In [1]:
# 사용 상의 편의를 위한 Initial Setting 

# 실행결과를 한 창에 표시
# InteractiveShell.ast_node_interactivity : 'all' | 'last' | 'last_expr' | 'none' (기본값은 'last_expr')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# numpy 소숫점 setting
import numpy as np
np.set_printoptions(precision=7)

# pandas이용하여 grid display
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)         # 최대 표시 줄 수 제한 해제
pd.set_option('display.max_columns', None)  # 최대 표시 컬럼 수 제한 해제
pd.set_option('display.max_colwidth', -1)        # 컬럼내 데이터 표시 제한 해제

# Warning Message ignore
import warnings
# warnings.filterwarnings(action='default')
warnings.filterwarnings(action='ignore')

In [2]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.1'

### Data 전처리

#### Beyond Good and Evil (Friedrich Nietzsche)
https://www.marxists.org/reference/archive/nietzsche/1886/beyond-good-evil/index.htm

In [3]:
# Corpus(말뭉치)를 다운로드 받아 모두 소문자로 변환
# 해당 Corpus는 니체가 1886년에 쓴 'Beyond Good and Evil' 임

import keras
import numpy as np

# 아래의 url에서 Corpus(말뭉치)를 다운로드 받음
path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

# 해당 file을 open하여 소문자로 변환 후 text 에 저장
text = open(path).read().lower()
print('corpus size :', len(text))

corpus size : 600893


In [4]:
# 해당 file 위치와 text 변수 type 확인
path
type(text)
len(text)
text[0:100]

'/home/ubuntu/.keras/datasets/nietzsche.txt'

str

600893

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all ph'

그 다음 `maxlen` 길이를 가진 시퀀스를 중복하여 추출합니다. 추출된 시퀀스를 원-핫 인코딩으로 변환하고 크기가 `(sequences, maxlen, unique_characters)`인 3D 넘파이 배열 `x`로 합칩니다. 동시에 훈련 샘플에 상응하는 타깃을 담은 배열 `y`를 준비합니다. 타깃은 추출된 시퀀스 다음에 오는 원-핫 인코딩된 글자입니다.

In [5]:
# maxlen 길이를 가진 sequence를 중복하여 추출
# 추출된 sequence를 one-hot encoding
# numpy Array x로 concatenate : (sequence, maxlen, unique_character)

# 60개 글자로 된 sequence추출
maxlen = 60

# 세 글자씩 건너 뛰면서 새로운 sequence를 sampling
step = 3

# 추출한 sequence를 담을 list
sentences = []

# sequence 다음 글자를 을 담을 list
next_chars = []


for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of Sequence:', len(sentences))

# Corpus에서 고유한 글자를 담은 list
chars = sorted(list(set(text)))
print('Unique letters :', len(chars))

# chars list에 있는 글자 및 해당 글자의 index를 mapping 한 dictionary
char_indices = dict((char, chars.index(char)) for char in chars)

# 글자를 0과 1의 이진 배열로 변경 (one-hot encoding)
print('making vector')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1   

Number of Sequence: 200278
Unique letters : 57
making vector


In [6]:
# 변수 확인
len(sentences)
len(next_chars)
sentences[0:10]
next_chars[0:10]
x.shape
y.shape

200278

200278

['preface\n\n\nsupposing that truth is a woman--what then? is the',
 'face\n\n\nsupposing that truth is a woman--what then? is there ',
 'e\n\n\nsupposing that truth is a woman--what then? is there not',
 '\nsupposing that truth is a woman--what then? is there not gr',
 'pposing that truth is a woman--what then? is there not groun',
 'sing that truth is a woman--what then? is there not ground\nf',
 'g that truth is a woman--what then? is there not ground\nfor ',
 'hat truth is a woman--what then? is there not ground\nfor sus',
 ' truth is a woman--what then? is there not ground\nfor suspec',
 'uth is a woman--what then? is there not ground\nfor suspectin']

['r', 'n', ' ', 'o', 'd', 'o', 's', 'p', 't', 'g']

(200278, 60, 57)

(200278, 57)

In [7]:
# 변수 확인
x[0]
y[0]
char_indices

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 ';': 21,
 '=': 22,
 '?': 23,
 '[': 24,
 ']': 25,
 '_': 26,
 'a': 27,
 'b': 28,
 'c': 29,
 'd': 30,
 'e': 31,
 'f': 32,
 'g': 33,
 'h': 34,
 'i': 35,
 'j': 36,
 'k': 37,
 'l': 38,
 'm': 39,
 'n': 40,
 'o': 41,
 'p': 42,
 'q': 43,
 'r': 44,
 's': 45,
 't': 46,
 'u': 47,
 'v': 48,
 'w': 49,
 'x': 50,
 'y': 51,
 'z': 52,
 'ä': 53,
 'æ': 54,
 'é': 55,
 'ë': 56}

## 네트워크 구성

이 네트워크는 하나의 `LSTM` 층과 그 뒤에 `Dense` 분류기가 뒤따릅니다. 분류기는 가능한 모든 글자에 대한 소프트맥스 출력을 만듭니다. 순환 신경망이 시퀀스 데이터를 생성하는 유일한 방법은 아닙니다. 최근에는 1D 컨브넷도 이런 작업에 아주 잘 들어 맞는다는 것이 밝혀졌습니다.

In [8]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

타깃이 원-핫 인코딩되어 있기 때문에 모델을 훈련하기 위해 `categorical_crossentropy` 손실을 사용합니다:

In [9]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## 언어 모델 훈련과 샘플링

훈련된 모델과 시드로 쓰일 간단한 텍스트가 주어지면 다음과 같이 반복하여 새로운 텍스트를 생성할 수 있습니다.

1.	지금까지 생성된 텍스트를 주입하여 모델에서 다음 글자에 대한 확률 분포를 뽑습니다.
2.	특정 온도로 이 확률 분포의 가중치를 조정합니다.
3.	가중치가 조정된 분포에서 무작위로 새로운 글자를 샘플링합니다.
4.	새로운 글자를 생성된 텍스트의 끝에 추가합니다.

다음 코드는 모델에서 나온 원본 확률 분포의 가중치를 조정하고 새로운 글자의 인덱스를 추출합니다(샘플링 함수입니다):

In [10]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

마지막으로 다음 반복문은 반복적으로 훈련하고 텍스트를 생성합니다. 에포크마다 학습이 끝난 후 여러가지 온도를 사용해 텍스트를 생성합니다. 이렇게 하면 모델이 수렴하면서 생성된 텍스트가 어떻게 진화하는지 볼 수 있습니다. 온도가 샘플링 전략에 미치는 영향도 보여 줍니다.

In [12]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

# 20 에포크 동안 모델을 훈련합니다
for epoch in range(1, 20):           
    print('에포크', epoch)
    # 데이터에서 한 번만 반복해서 모델을 학습합니다
    model.fit(x, y, batch_size=128, epochs=1)

    # 무작위로 시드 텍스트를 선택합니다
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    # 여러가지 샘플링 온도를 시도합니다
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        # 시드 텍스트에서 시작해서 400개의 글자를 생성합니다
        for i in range(400):
            # 지금까지 생성된 글자를 원-핫 인코딩으로 바꿉니다
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # 다음 글자를 샘플링합니다
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
Epoch 1/1
200278/200278 [==============================] - 216s 1ms/step - loss: 1.4476


--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the same exception of the spirit of the world the same the same the strenglation, and the sense of the spirit of the contrary and subtle the same to the spirit of the same to an all the same to sublime and the more and the sense of the sense of the sense of the strenglate and all the the same to an actions and subtle the same to an all the same to the more an and the sense of the strenglates and
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for a the exercient, and the rest of entire in the rule and disclessing that of explanationation, when the
fact and share be in order to
one consist of the more feild the morality of the contemporary and an its for acts and society and man is attain the greater provery and religious and towards are only degree of the streng of the soul, how man of his despessible be for whi

--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the strength that it is the man and strength the suffered the consequently and superiority to the spirit and pather the strength and the strength and constinction and the strength and superiority and strength and the most strength and superiority of the strength and strong and the strength of the sense of the strength the strength and powerful and all the spirit and conscience of the strength to
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through former of the threates and decals all the present to the finer and still perhaps it
is such a thing of nature and impirine and iflent to the read about the philosophy the whole findly and roman the same and point the educed of the consequently as a man would he would have our falses of suffered and faith and when it had to the will to have a mediocres in the first of the gr

--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the sense of the sufferer of the soul the sense of the strength and the sense of the property and the stronger of the strength to himself--who is a soul of the strength, and the belief in the conditions and the sense of the strength to the strength of the senses to any profounds of the religious the sense of the strength, and the surprised the stronger the soul the subjection and interpretation 
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for their conducted with themselves for the characteristic suffered with relation of his moral man the morality and are
fair for the surprised a metaphysical, and when he sought to the
intellest and for the sense of the spirits of conditions and personal nature of the deperient and man of the surprises and proved to men in the deeped to himself--who concernsning the accordi

--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the most and the most pressions of the development of the most and the domain of the most and soul is the most state of the most and the master and the souls and the most pressed and the respect to the discipline and contempt to the same and the fact of the such a man and the most conscience of the conscience of the most and the same artists of the present and the contempt of the expression and 
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for for the most secret and sense of the case of the entire of their spirit and love and things the supposed a conscience of the present and taste of the souls and a scheal believes to generally say, our explanable and discipline to means of the great and the domains of the sensible of a moral in the meass and states of all an interestion of the explanation and contempt and

...

--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through formul in the sense of the world in the sense of the sense of the sense of the sense of the sense of the world and the contradict to the conscience of the sense of the sense the sense of the sense in the sense of the sense and the man in the sense of the sense and well as a profound the same the sense of the structive the sense of the sense of the sense and the sense of the sense of the sense of the
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through formed to the believe itself, and the some superstange and person as the destructive and powerful prevailed at all, the proper and interested who have it the vainism for the soul sport of antiquity of the former them believe by the sense itself the mals world to assumes because one in himself and according to the present will man and explanation, and for the sense even in t

--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the soul and the sense of the most satisfance of the same and the sense of the sense of the fact that the sense of such and who see the will to the sense of the sense of the sense of the sense of the strength of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the strength of the strength of the world a
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through formerly desire the conscience of the serious spirit in the belief in the most sometiy of the sense of the pressols and short of the reason and which may and the rest of thinked to many and belong there are everything most satisfance of sublime in the contrary and artistly the arteness of the strength of the former and faith and the betoranish in the faculty and lives the m

--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the strong of the state of the contempt of the strong of the sense of the same of the state of strong and state of the same of the strong of the same standard and states of the same standard and ancient strong and the sense of the standard and also and the contempt of the same and and the states of the same of the same of the same of the most satisfaction of the state of the sense of the same st
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the events like the mark, and really with injury of the others of the soul speak of the sense and pure over the restruction of merely conscience and literation, and the precisely the reason and also the word of a speaks, even an ancervers of his disclose and hard of the spirit, the reals and away and exten volint of so free reprecistection in the sense of the good and h

--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the spirit it is always and the most and self and the superiority of the superiority of the supersative and the problem of the supersation of the same the superiority of the superiority, in the same the condition of the superiority of the presentiment of all all the last and the presentiment and something and the strong and will to the still sense of the same and something and all the spirit his
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for anceols and self stronger of the exception, without all the extince of art of woman, "the condition of the sublime the most sense in the power to the strong its religion, which all our subtlest, in the most intellectuality of which is always the still all the supersation is a man and person of the indevelops of the human and the most person the enough of the onnession o

--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the soul the morality of the standpoint to the problem of the standpoint to the same the thing to the standpoint to the same spirit the man is the same morality and soul is to be the morality of the strength of the standpoint to the standpoint of the standpoint to a metaphysical interrition of the strenathes to the same men, the standpoint to the problem of the standpoint to the same and the str
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through fore in the presence, as we are the danger, in the most interlogation of the reality religion become of the fact that which the account to the sole suffering of science is not the facts of his former of the ward in all the and latter all the thing idea in the called the sense of their own word, the fact that when it has not in the same desire and the canorful, and when it m

여기서 볼 수 있듯이 낮은 온도는 아주 반복적이고 예상되는 텍스트를 만듭니다. 하지만 국부적인 구조는 매우 실제와 같습니다. 특히 모든 단어(단어는 글자의 지역 패턴으로 이루어집니다)가 실제 영어 단어입니다. 높은 온도에서 생성된 텍스트는 아주 흥미롭고 놀라우며 창의적이기도 합니다. 이따금 꽤 그럴싸하게 보이는 완전히 새로운 단어를 창조합니다(‘begarmed’와 ‘isharent’ 같은 단어입니다). 높은 온도에서는 국부적인 구조가 무너지기 시작합니다. 대부분의 단어가 어느정도 무작위한 문자열로 보입니다. 확실히 이 네트워크에서는 텍스트 생성에 가장 좋은 온도는 0.5입니다. 항상 다양한 샘플링 전략으로 실험해 봐야합니다! 학습된 구조와 무작위성 사이에 균형을 잘 맞추면 흥미로운 것을 만들 수 있습니다.

더 많은 데이터에서 크고 깊은 모델을 훈련하면 이것보다 훨씬 논리적이고 실제와 같은 텍스트 샘플을 생성할 수 있습니다. 당연히 우연이 아닌 의미 있는 텍스트가 생성된다고 기대하지 마세요. 글자를 연속해서 나열하기 위한 통계 모델에서 데이터를 샘플링한 것뿐입니다. 언어는 의사소통의 수단입니다. 의사소통이 의미하는 것과 의사소통이 인코딩된 메시지의 통계 구조 사이는 차이가 있습니다. 이 차이를 검증하기 위해 다음과 같은 사고 실험을 해보죠. 컴퓨터가 대부분의 디지털 통신에서 하는 것처럼 사람의 언어가 의사소통을 압축하는데 더 뛰어나다면 어떨까요? 언어의 의미가 줄진 않지만 고유한 통계 구조가 사라질 것입니다. 이는 방금과 같은 언어 모델을 학습하는 것을 불가능하게 만듭니다.

## 정리

* 이전의 토큰이 주어지면 다음 토큰(들)을 예측하는 모델을 훈련하여 시퀀스 데이터를 생성할 수 있습니다.
* 텍스트의 경우 이런 모델을 언어 모델이라 부릅니다. 단어 또는 글자 단위 모두 가능합니다.
* 다음 토큰을 샘플링할 때 모델이 만든 출력에 집중하는 것과 무작위성을 주입하는 것 사이에 균형을 맞추어야 합니다.
* 이를 위해 소프트맥스 온도 개념을 사용합니다. 항상 다양한 온도를 실험해서 적절한 값을 찾습니다.